## Introduction 

Pelajaran ini akan membahas: 
- Apa itu pemanggilan fungsi dan kasus penggunaannya 
- Cara membuat pemanggilan fungsi menggunakan OpenAI 
- Cara mengintegrasikan pemanggilan fungsi ke dalam aplikasi 

## Learning Goals 

Setelah menyelesaikan pelajaran ini Anda akan mengetahui cara dan memahami: 

- Tujuan menggunakan pemanggilan fungsi 
- Menyiapkan Pemanggilan Fungsi menggunakan Layanan OpenAI 
- Merancang pemanggilan fungsi yang efektif untuk kasus penggunaan aplikasi Anda 


## Memahami Panggilan Fungsi

Untuk pelajaran ini, kami ingin membangun fitur untuk startup pendidikan kami yang memungkinkan pengguna menggunakan chatbot untuk menemukan kursus teknis. Kami akan merekomendasikan kursus yang sesuai dengan tingkat keterampilan mereka, peran saat ini, dan teknologi yang diminati.

Untuk menyelesaikan ini, kami akan menggunakan kombinasi:
 - `OpenAI` untuk membuat pengalaman chat bagi pengguna
 - `Microsoft Learn Catalog API` untuk membantu pengguna menemukan kursus berdasarkan permintaan pengguna
 - `Function Calling` untuk mengambil kueri pengguna dan mengirimkannya ke fungsi untuk membuat permintaan API.

Untuk memulai, mari kita lihat mengapa kita ingin menggunakan function calling sejak awal:

print("Pesan dalam permintaan berikutnya:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dapatkan respons baru dari GPT di mana ia dapat melihat respons fungsi


print(second_response.choices[0].message)


### Mengapa Pemanggilan Fungsi

Jika Anda telah menyelesaikan pelajaran lain dalam kursus ini, Anda mungkin sudah memahami kekuatan menggunakan Large Language Models (LLM). Semoga Anda juga dapat melihat beberapa keterbatasan mereka.

Pemanggilan Fungsi adalah fitur dari Layanan OpenAI yang dirancang untuk mengatasi tantangan berikut:

Format Respons yang Tidak Konsisten:
- Sebelum pemanggilan fungsi, respons dari model bahasa besar tidak terstruktur dan tidak konsisten. Pengembang harus menulis kode validasi yang kompleks untuk menangani setiap variasi dalam output.

Integrasi Terbatas dengan Data Eksternal:
- Sebelum fitur ini, sulit untuk menggabungkan data dari bagian lain aplikasi ke dalam konteks obrolan.

Dengan menstandarisasi format respons dan memungkinkan integrasi mulus dengan data eksternal, pemanggilan fungsi menyederhanakan pengembangan dan mengurangi kebutuhan logika validasi tambahan.

Pengguna tidak bisa mendapatkan jawaban seperti "Bagaimana cuaca saat ini di Stockholm?". Ini karena model terbatas pada waktu data dilatih.

Mari kita lihat contoh di bawah yang menggambarkan masalah ini:

Misalkan kita ingin membuat basis data data siswa agar kita dapat menyarankan kursus yang tepat untuk mereka. Di bawah ini kita memiliki dua deskripsi siswa yang sangat mirip dalam data yang mereka miliki.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Kami ingin mengirim ini ke LLM untuk mengurai data. Ini nantinya dapat digunakan dalam aplikasi kami untuk mengirim ini ke API atau menyimpannya dalam basis data.

Mari buat dua prompt identik yang kami instruksikan kepada LLM tentang informasi apa yang kami minati:


Kami ingin mengirim ini ke LLM untuk menguraikan bagian-bagian yang penting bagi produk kami. Jadi kami dapat membuat dua prompt identik untuk menginstruksikan LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Setelah membuat dua prompt ini, kita akan mengirimkannya ke LLM dengan menggunakan `openai.ChatCompletion`. Kita menyimpan prompt tersebut dalam variabel `messages` dan menetapkan peran sebagai `user`. Ini untuk meniru pesan dari pengguna yang ditulis ke chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Sekarang kita dapat mengirim kedua permintaan ke LLM dan memeriksa respons yang kita terima.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Meskipun prompt-nya sama dan deskripsinya mirip, kita bisa mendapatkan format properti `Grades` yang berbeda.

Jika Anda menjalankan sel di atas beberapa kali, formatnya bisa berupa `3.7` atau `3.7 GPA`.

Ini karena LLM mengambil data tidak terstruktur dalam bentuk prompt tertulis dan juga mengembalikan data tidak terstruktur. Kita perlu memiliki format terstruktur agar kita tahu apa yang diharapkan saat menyimpan atau menggunakan data ini.

Dengan menggunakan pemanggilan fungsi, kita dapat memastikan bahwa kita menerima data terstruktur kembali. Saat menggunakan pemanggilan fungsi, LLM sebenarnya tidak memanggil atau menjalankan fungsi apa pun. Sebaliknya, kita membuat struktur untuk diikuti LLM dalam responsnya. Kemudian kita menggunakan respons terstruktur tersebut untuk mengetahui fungsi apa yang harus dijalankan dalam aplikasi kita.


![Diagram Alur Pemanggilan Fungsi](../../../../translated_images/id/Function-Flow.083875364af4f4bb.webp)


Kita kemudian dapat mengambil apa yang dikembalikan dari fungsi tersebut dan mengirimkannya kembali ke LLM. LLM kemudian akan merespons menggunakan bahasa alami untuk menjawab pertanyaan pengguna.


### Kasus Penggunaan untuk menggunakan pemanggilan fungsi

**Memanggil Alat Eksternal**  
Chatbot sangat baik dalam memberikan jawaban atas pertanyaan dari pengguna. Dengan menggunakan pemanggilan fungsi, chatbot dapat menggunakan pesan dari pengguna untuk menyelesaikan tugas tertentu. Misalnya, seorang siswa dapat meminta chatbot untuk "Kirim email ke instruktur saya mengatakan saya membutuhkan bantuan lebih dengan mata pelajaran ini". Ini dapat membuat panggilan fungsi ke `send_email(to: string, body: string)`

**Membuat Query API atau Database**  
Pengguna dapat menemukan informasi menggunakan bahasa alami yang diubah menjadi query atau permintaan API yang terformat. Contohnya bisa seorang guru yang meminta "Siapa saja siswa yang menyelesaikan tugas terakhir" yang dapat memanggil fungsi bernama `get_completed(student_name: string, assignment: int, current_status: string)`

**Membuat Data Terstruktur**  
Pengguna dapat mengambil blok teks atau CSV dan menggunakan LLM untuk mengekstrak informasi penting darinya. Misalnya, seorang siswa dapat mengubah artikel Wikipedia tentang perjanjian damai untuk membuat kartu flash AI. Ini dapat dilakukan dengan menggunakan fungsi yang disebut `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Membuat Panggilan Fungsi Pertama Anda

Proses membuat panggilan fungsi mencakup 3 langkah utama:
1. Memanggil API Chat Completions dengan daftar fungsi Anda dan pesan pengguna
2. Membaca respons model untuk melakukan tindakan yaitu menjalankan fungsi atau Panggilan API
3. Membuat panggilan lain ke API Chat Completions dengan respons dari fungsi Anda untuk menggunakan informasi tersebut dalam membuat respons kepada pengguna.


![Alur Panggilan Fungsi](../../../../translated_images/id/LLM-Flow.3285ed8caf4796d7.webp)


### Elemen dari sebuah panggilan fungsi

#### Input Pengguna

Langkah pertama adalah membuat pesan pengguna. Ini dapat ditetapkan secara dinamis dengan mengambil nilai dari input teks atau Anda dapat menetapkan nilai di sini. Jika ini adalah pertama kalinya Anda bekerja dengan Chat Completions API, kita perlu mendefinisikan `role` dan `content` dari pesan tersebut.

`role` dapat berupa `system` (membuat aturan), `assistant` (model) atau `user` (pengguna akhir). Untuk pemanggilan fungsi, kita akan menetapkannya sebagai `user` dan sebuah contoh pertanyaan.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Membuat fungsi.

Selanjutnya kita akan mendefinisikan sebuah fungsi dan parameter dari fungsi tersebut. Kita akan menggunakan hanya satu fungsi di sini yang disebut `search_courses` tetapi Anda dapat membuat beberapa fungsi.

**Penting** : Fungsi-fungsi disertakan dalam pesan sistem ke LLM dan akan termasuk dalam jumlah token yang tersedia yang Anda miliki.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisi** 

Struktur definisi fungsi memiliki beberapa tingkat, masing-masing dengan properti sendiri. Berikut adalah rincian struktur bersarangnya:

**Properti Fungsi Tingkat Atas:**

`name` - Nama fungsi yang ingin kita panggil. 

`description` - Ini adalah deskripsi tentang bagaimana fungsi bekerja. Di sini penting untuk spesifik dan jelas 

`parameters` - Daftar nilai dan format yang ingin Anda model hasilkan dalam responsnya 

**Properti Objek Parameter:**

`type` - Tipe data dari objek parameter (biasanya "object")

`properties` - Daftar nilai spesifik yang akan digunakan model untuk responsnya 

**Properti Parameter Individu:**

`name` - Didefinisikan secara implisit oleh kunci properti (misalnya, "role", "product", "level")

`type` - Tipe data dari parameter spesifik ini (misalnya, "string", "number", "boolean") 

`description` - Deskripsi dari parameter spesifik tersebut 

**Properti Opsional:**

`required` - Array yang mencantumkan parameter mana yang diperlukan agar pemanggilan fungsi dapat diselesaikan 


### Membuat panggilan fungsi  
Setelah mendefinisikan sebuah fungsi, sekarang kita perlu memasukkannya dalam panggilan ke API Chat Completion. Kita melakukan ini dengan menambahkan `functions` ke permintaan. Dalam kasus ini `functions=functions`.  

Ada juga opsi untuk mengatur `function_call` ke `auto`. Ini berarti kita akan membiarkan LLM memutuskan fungsi mana yang harus dipanggil berdasarkan pesan pengguna daripada menetapkannya sendiri.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sekarang mari kita lihat responsnya dan bagaimana formatnya:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Anda dapat melihat bahwa nama fungsi dipanggil dan dari pesan pengguna, LLM dapat menemukan data yang sesuai dengan argumen fungsi tersebut.


## 3.Mengintegrasikan Panggilan Fungsi ke dalam Aplikasi. 


Setelah kita menguji respons yang diformat dari LLM, sekarang kita dapat mengintegrasikannya ke dalam sebuah aplikasi. 

### Mengelola alur 

Untuk mengintegrasikan ini ke dalam aplikasi kita, mari ambil langkah-langkah berikut: 

Pertama, mari buat panggilan ke layanan OpenAI dan simpan pesan dalam variabel yang disebut `response_message`. 


In [ ]:
response_message = response.choices[0].message

Sekarang kita akan mendefinisikan fungsi yang akan memanggil API Microsoft Learn untuk mendapatkan daftar kursus:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Sebagai praktik terbaik, kita kemudian akan melihat apakah model ingin memanggil sebuah fungsi. Setelah itu, kita akan membuat salah satu fungsi yang tersedia dan mencocokkannya dengan fungsi yang sedang dipanggil.  
Kita kemudian akan mengambil argumen dari fungsi tersebut dan memetakan mereka ke argumen dari LLM.

Terakhir, kita akan menambahkan pesan panggilan fungsi dan nilai-nilai yang dikembalikan oleh pesan `search_courses`. Ini memberikan LLM semua informasi yang dibutuhkan untuk  
menanggapi pengguna menggunakan bahasa alami.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Sekarang kami akan mengirim pesan yang diperbarui ke LLM sehingga kami dapat menerima respons dalam bahasa alami, bukan respons yang diformat JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Tantangan Kode

Kerja bagus! Untuk melanjutkan pembelajaran Anda tentang OpenAI Function Calling, Anda dapat membangun: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Lebih banyak parameter dari fungsi yang mungkin membantu pembelajar menemukan lebih banyak kursus. Anda dapat menemukan parameter API yang tersedia di sini:  
 - Buat panggilan fungsi lain yang mengambil lebih banyak informasi dari pembelajar seperti bahasa asli mereka  
 - Buat penanganan kesalahan ketika panggilan fungsi dan/atau panggilan API tidak mengembalikan kursus yang sesuai  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk akurasi, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang sahih. Untuk informasi penting, disarankan menggunakan terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau salah tafsir yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
